In [1]:
import numpy as np
import pandas as pd

SELECT statements

In [2]:
lines = '''-- insert into tblCITY
-- insert into tblVENUE
-- insert into tblCONCERT
-- insert into tblCUSTOMER
-- insert into tblTICKET
-- insert into tblPRODUCT_PRICE_HISTORY
-- insert into tblTICKET_PRODUCT
-- insert into tblPRODUCT
-- insert into tblARTIST
-- insert into tblARTIST_SONG
-- insert into tblSONG
-- insert into tblSONG_ALBUM
-- insert into tblALBUM
-- insert into tblPRODUCER_ALBUM
-- insert into tblARTIST_LABEL
-- insert into tblLABEL
-- insert into tblSONG_LINEUP
-- insert into tblSONG_GENRE
-- insert into tblCONCERT_EMPLOYEE
-- insert into tblEMPLOYEE
-- insert into tblEMPLOYEE_STATUS'''

In [4]:
for line in lines.split('\n'):
    table = line.strip().split(' ')[-1]
    print(f'SELECT * FROM {table}')

SELECT * FROM tblCITY
SELECT * FROM tblVENUE
SELECT * FROM tblCONCERT
SELECT * FROM tblCUSTOMER
SELECT * FROM tblTICKET
SELECT * FROM tblPRODUCT_PRICE_HISTORY
SELECT * FROM tblTICKET_PRODUCT
SELECT * FROM tblPRODUCT
SELECT * FROM tblARTIST
SELECT * FROM tblARTIST_SONG
SELECT * FROM tblSONG
SELECT * FROM tblSONG_ALBUM
SELECT * FROM tblALBUM
SELECT * FROM tblPRODUCER_ALBUM
SELECT * FROM tblARTIST_LABEL
SELECT * FROM tblLABEL
SELECT * FROM tblSONG_LINEUP
SELECT * FROM tblSONG_GENRE
SELECT * FROM tblCONCERT_EMPLOYEE
SELECT * FROM tblEMPLOYEE
SELECT * FROM tblEMPLOYEE_STATUS


Stored Procedures

In [46]:
# table : string
# fks : [[[[fkname1, datatype1], [fkname2, datatype2] ... ], fkid, table]]
# cols: [[colname, datatype]]
def write_sproc(srctable, fks, cols):
    print(f'-- insert into {srctable}')
    # > create base name
    code = f'GO\nCREATE PROCEDURE INSERT_{srctable}\n'
    # > add cols
    for fklist, fkid, table in fks:
        for fkname, datatype in fklist:
            code += f'@{fkname} {datatype},\n'
    for colname, datatype in cols:
        code += f'@{colname} {datatype},\n'
    code = code.rstrip()[:-1] + '\n'
    code += 'AS\n'
    # > declare variables
    code += 'DECLARE '
    for fklist, fkid, table in fks:
        code += f'@{table[3:]}_ID INT, '
    code = code[:-2] + '\n'
    # > add foreign key lookups
    for fklist, fkid, table in fks:
        # >> get the query statement
        code += f'-- querying for {fkid} from {table}\n'
        fknames = [fkitem[0] for fkitem in fklist]
        query = f'SET @{table[3:]}_ID = (SELECT {fkid} FROM {table} WHERE '
        for fkname in fknames:
            query += f'{fkname} = @{fkname} AND '
        if(query[-4:] == 'AND '):
            query = query[:-4]
        query = query[:-1] + ')\n'
        code += query
        # >> error handling
        handling = f'IF @{table[3:]}_ID IS NULL\n'
        handling += f'BEGIN\n    PRINT \'ERROR @{table[3:]}_ID IS NULL\';\n'
        handling += f'    THROW 500001, \'NULL ID\', 1;\n'
        handling += 'END\n'
        code += handling
    # > do actual transaction
    code += '-- conducting actual insert statement\n'
    code += f'BEGIN TRAN T1\nINSERT INTO {srctable} ('
    # >> add on the foreign keys
    for fklist, fkid, table in fks:
        code += f'{fkid}, '
    # >> add on the columns
    for colname, datatype in cols:
        code += f'{colname}, '
    code = code[:-2] + ')\n'
    # >> add values for foreign keys
    code += f'VALUES ('
    for fklist, fkid, table in fks:
        code += f'@{table[3:]}_ID, '
    # >> add values for columns
    for colname, datatype in cols:
        code += f'@{colname}, '
    code = code[:-2] + ')\n'
    # >> error handling
    code += 'IF @@ERROR <> 0\n    ROLLBACK TRAN T1\nELSE\n    COMMIT TRAN T1\n'
    # > add final go
    code += 'GO\n'
    return code

In [48]:
params_list = []
# > tblCITY
table = 'tblCITY'
fks = [[[['CountryName','VARCHAR(50)']],'CountryID','tblCOUNTRY']]
cols = [['CityName','VARCHAR(50)'],['CityDescription', 'VARCHAR(250) NULL']]
params_list.append([table, fks, cols])
# > tblVENUE
table = 'tblVENUE'
fks = [[[['CityName','VARCHAR(50)']],'CityID','tblCITY'],[[['VenueTypeName','VARCHAR(50)']],'VenueTypeID','tblVENUE_TYPE']]
cols = [['VenueName','VARCHAR(50)'],['VenueDescription', 'VARCHAR(250) NULL']]
params_list.append([table, fks, cols])
# > tblCONCERT
table = 'tblCONCERT'
fks = [[[['ConcertTypeName','VARCHAR(50)']],'ConcertTypeID','tblCONCERT_TYPE'],[[['VenueName','VARCHAR(50)']],'VenueID','tblVENUE']]
cols = [['ConcertName','VARCHAR(50)'],['ConcertDescription', 'VARCHAR(250) NULL']]
params_list.append([table, fks, cols])
# > tblCUSTOMER
table = 'tblCUSTOMER'
fks = [[[['CustomerTypeName','VARCHAR(50)']],'CustomerTypeID','tblCUSTOMER_TYPE']]
cols = [['CustomerFname','VARCHAR(50)'],['CustomerLname','VARCHAR(50)'],['CustomerDOB', 'DATE']]
params_list.append([table, fks, cols])
# > tblTICKET
table = 'tblTICKET'
fks = [[[['CustomerFname','VARCHAR(50)'],['CustomerLname','VARCHAR(50)'],['CustomerDOB', 'DATE']],'CustomerID','tblCUSTOMER'],[[['TicketTypeName','VARCHAR(50)']],'TicketTypeID','tblTICKET_TYPE'],[[['ConcertName','VARCHAR(50)']],'ConcertID','tblCONCERT']]
cols = [['TicketName','VARCHAR(50)'],['Price','NUMERIC(10,2)']]
params_list.append([table, fks, cols])
# > tblPRODUCT_PRICE_HISTORY
table = 'tblPRODUCT_PRICE_HISTORY'
fks = [[[['ProductName','VARCHAR(50)']],'ProductID','tblPRODUCT']]
cols = [['Price','NUMERIC(4,2)'],['BeginDate','DATE'],['EndDate','DATE']]
params_list.append([table, fks, cols])
# > tblTICKET_PRODUCT
table = 'tblTICKET_PRODUCT'
fks = [[[['TicketName','VARCHAR(50)']],'TicketID','tblTICKET'],[[['ProductName','VARCHAR(50)']],'ProductID','tblPRODUCT']]
cols = [['Quantity','INT']]
params_list.append([table, fks, cols])
# > tblPRODUCT
table = 'tblPRODUCT'
fks = [[[['ProductTypeName','VARCHAR(50)']],'ProductTypeID','tblPRODUCT_TYPE']]
cols = [['ProductName','VARCHAR(50)']]
params_list.append([table, fks, cols])
# > tblARTIST_SONG
table = 'tblARTIST_SONG'
fks = [[[['ArtistFname','VARCHAR(50)'],['ArtistLname','VARCHAR(50)'],['ArtistDOB', 'DATE']],'ArtistID','tblARTIST'],[[['SongName','VARCHAR(50)']],'SongID','tblSONG']]
cols = []
params_list.append([table, fks, cols])
# > tblSONG_ALBUM
table = 'tblSONG_ALBUM'
fks = [[[['SongName','VARCHAR(50)']],'SongID','tblSONG'],[[['AlbumName','VARCHAR(50)']],'AlbumID','tblALBUM']]
cols = []
params_list.append([table, fks, cols])
# > tblALBUM
table = 'tblALBUM'
fks = [[[['AlbumTypeName','VARCHAR(50)']],'AlbumTypeID','tblALBUM_TYPE']]
cols = [['AlbumName','VARCHAR(50)'],['AlbumDescription','VARCHAR(250) NULL'],['AlbumReleaseDate','DATE']]
params_list.append([table, fks, cols])
# > tblPRODUCER_ALBUM
table = 'tblPRODUCER_ALBUM'
fks = [[[['ProducerFname','VARCHAR(50)'],['ProducerLname','VARCHAR(50)'],['ProducerDOB','DATE']],'ProducerID','tblPRODUCER'],[[['AlbumName','VARCHAR(50)']],'AlbumID','tblALBUM']]
cols = []
params_list.append([table, fks, cols])
# > tblARTIST_LABEL
table = 'tblARTIST_LABEL'
fks = [[[['ArtistFname','VARCHAR(50)'],['ArtistLname','VARCHAR(50)'],['ArtistDOB', 'DATE']],'ArtistID','tblARTIST'],[[['LabelName','VARCHAR(50)']],'LabelID','tblLABEL']]
cols = [['BeginDate','DATE'],['EndDate','DATE']]
params_list.append([table, fks, cols])
# > tblSONG_LINEUP
table = 'tblSONG_LINEUP'
fks = [[[['ConcertName','VARCHAR(50)']],'ConcertID','tblCONCERT'],[[['SongName','VARCHAR(50)']],'SongID','tblSONG']]
cols = [['Duration','NUMERIC(8,4)']]
params_list.append([table, fks, cols])
# > tblSONG_GENRE
table = 'tblSONG_GENRE'
fks = [[[['SongName','VARCHAR(50)']],'SongID','tblSONG'],[[['GenreName','VARCHAR(50)']],'GenreID','tblGENRE']]
cols = []
params_list.append([table, fks, cols])
# > tblCONCERT_EMPLOYEE
table = 'tblCONCERT_EMPLOYEE'
fks = [[[['EmployeeFname','VARCHAR(50)'],['EmployeeLname','VARCHAR(50)'],['EmployeeDOB', 'DATE']],'EmployeeID','tblEMPLOYEE'],[[['EmployeeTypeName','VARCHAR(50)']],'EmployeeTypeID','tblEMPLOYEE_TYPE'],[[['EmployeeRoleName','VARCHAR(50)']],'EmployeeRoleID','tblEMPLOYEE_ROLE'],[[['ConcertName','VARCHAR(50)']],'ConcertID','tblCONCERT']]
cols = []
params_list.append([table, fks, cols])
# > tblEMPLOYEE_STATUS
table = 'tblEMPLOYEE_STATUS'
fks = [[[['EmployeeFname','VARCHAR(50)'],['EmployeeLname','VARCHAR(50)'],['EmployeeDOB', 'DATE']],'EmployeeID','tblEMPLOYEE'],[[['StatusName','VARCHAR(50)']],'StatusID','tblSTATUS']]
cols = [['BeginDate','DATE'],['EndDate','DATE']]
params_list.append([table, fks, cols])
# > print parameters
for params in params_list:
    print(write_sproc(*params))


-- insert into tblCITY
GO
CREATE PROCEDURE INSERT_tblCITY
@CountryName VARCHAR(50),
@CityName VARCHAR(50),
@CityDescription VARCHAR(250) NULL
AS
DECLARE @COUNTRY_ID INT
-- querying for CountryID from tblCOUNTRY
SET @COUNTRY_ID = (SELECT CountryID FROM tblCOUNTRY WHERE CountryName = @CountryName)
IF @COUNTRY_ID IS NULL
BEGIN
    PRINT 'ERROR @COUNTRY_ID IS NULL';
    THROW 500001, 'NULL ID', 1;
END
-- conducting actual insert statement
BEGIN TRAN T1
INSERT INTO tblCITY (CountryID, CityName, CityDescription)
VALUES (@COUNTRY_ID, @CityName, @CityDescription)
IF @@ERROR <> 0
    ROLLBACK TRAN T1
ELSE
    COMMIT TRAN T1
GO

-- insert into tblVENUE
GO
CREATE PROCEDURE INSERT_tblVENUE
@CityName VARCHAR(50),
@VenueTypeName VARCHAR(50),
@VenueName VARCHAR(50),
@VenueDescription VARCHAR(250) NULL
AS
DECLARE @CITY_ID INT, @VENUE_TYPE_ID INT
-- querying for CityID from tblCITY
SET @CITY_ID = (SELECT CityID FROM tblCITY WHERE CityName = @CityName)
IF @CITY_ID IS NULL
BEGIN
    PRINT 'ERROR @CITY_ID